# Build an Agent

By themselves, language models can't take actions - they just output text.
A big use case for LangChain is creating **agents**.
[Agents](/docs/concepts/agents) are systems that use [LLMs](/docs/concepts/chat_models) as reasoning engines to determine which actions to take and the inputs necessary to perform the action.
After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish. This is often achieved via [tool-calling](/docs/concepts/tool_calling).

In this tutorial we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it call the search tool, and have conversations with it.

## End-to-end agent

The code snippet below represents a fully functional agent that uses an LLM to decide which tools to use. It is equipped with a generic search tool. It has conversational memory - meaning that it can be used as a multi-turn chatbot.

In the rest of the guide, we will walk through the individual components and what each part does - but if you want to just grab some code and get started, feel free to use this!

In [7]:
import dotenv
import os

dotenv.load_dotenv()

# Import relevant functionality
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
# model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
model = ChatOpenAI(model_name='gpt-4o-mini')
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [9]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im chadi! and i live in byblos, lebanon")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi im chadi! and i live in byblos, lebanon
================================== Ai Message ==================================

Hi Chadi! It's great to meet you. Byblos is a beautiful city with a rich history. How can I assist you today?


In [34]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_iS19TIPJiSSD57eW4EgNBYjw)
 Call ID: call_iS19TIPJiSSD57eW4EgNBYjw
  Args:
    query: San Francisco weather
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weather2travel.com/california/san-francisco/september/", "content": "![](https://www.facebook.com/tr?id=237444546639682&ev=PageView&noscript=1)\n![weather2travel.com - travel deals for your holiday in the sun](/images/weather2travel-logo.svg)\n![Click to search](/images/search-white.png)\n\n# San Francisco weather in September 2025\n\nExpect  **daytime maximum temperatures of 22°C** in **San Francisco, California** in **September** based on long-term weather averages. There are **9 hours of sunshine per day**

## Setup

### Jupyter Notebook

This guide (and most of the other guides in the documentation) uses [Jupyter notebooks](https://jupyter.org/) and assumes the reader is as well. Jupyter notebooks are perfect interactive environments for learning how to work with LLM systems because oftentimes things can go wrong (unexpected output, API down, etc), and observing these cases is a great way to better understand building with LLMs.

This and other tutorials are perhaps most conveniently run in a Jupyter notebook. See [here](https://jupyter.org/install) for instructions on how to install.

### Installation

To install LangChain run:

In [37]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

  Using cached langchain_community-0.3.24-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain-0.3.25-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
Using cached langchain_community-0.3.24-py3-none-any.whl (2.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 359.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 kB 366.6 kB/s eta 0:00:00a 0:00:01
Using cached langchain-0.3.25-py3-none-any.whl (1.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 364.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 360.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.2/165.2 kB 372.8 kB/s eta 0:00:0000:0100:01
Using cached langchain_text_splitters-0.3.8-py3-none-any.whl (32 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.7/633.7 kB 363.9 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: zstandard
    Found

For more details, see our [Installation guide](/docs/how_to/installation).

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

Or, if in a notebook, you can set them with:

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

### Tavily

We will be using [Tavily](/docs/integrations/tools/tavily_search) (a search engine) as a tool.
In order to use it, you will need to get and set an API key:

```bash
export TAVILY_API_KEY="..."
```

Or, if in a notebook, you can set it with:

```python
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()
```

## Define tools

We first need to create the tools we want to use. Our main tool of choice will be [Tavily](/docs/integrations/tools/tavily_search) - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.


In [41]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weather2travel.com/california/san-francisco/september/', 'content': '![](https://www.facebook.com/tr?id=237444546639682&ev=PageView&noscript=1)\n![weather2travel.com - travel deals for your holiday in the sun](/images/weather2travel-logo.svg)\n![Click to search](/images/search-white.png)\n\n# San Francisco weather in September 2025\n\nExpect  **daytime maximum temperatures of 22°C** in **San Francisco, California** in **September** based on long-term weather averages. There are **9 hours of sunshine per day** on average. [...] ![12 hours of daylight per day in September in San Francisco](/images_symbols/daylight12.png)\n![No heat & humidity in September in San Francisco](/images_symbols/heathumidity0.png)\n![7 mm monthly rainfall in September in San Francisco](/images_symbols/rainfall1.png)\n![UV (maximum) index 7 (High) in September in San Francisco](/images_symbols/uv5.png)\n![14°C sea temperature in September in San Francisco](/images_symbols/sea-temperature6.p

## Using Language Models

Next, let's learn how to use a language model to call tools. LangChain supports many different language models that you can use interchangably - select the one you want to use below!

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs overrideParams={{openai: {model: "gpt-4"}}} />


In [3]:
# | output: false
# | echo: false

from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-sonnet-20240229")

You can call the language model by passing in a list of messages. By default, the response is a `content` string.

In [44]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

We can now see what it is like to enable this model to do tool calling. In order to enable that we use `.bind_tools` to give the language model knowledge of these tools

In [47]:
model_with_tools = model.bind_tools(tools)

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the `content` field as well as the `tool_calls` field.

In [50]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


Now, let's try calling it with some input that would expect a tool to be called.

In [56]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather'}, 'id': 'call_ZRezRxAxpsnjkI1b2KC5mEUq', 'type': 'tool_call'}]


We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

## Create the agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using [LangGraph](/docs/concepts/architecture/#langgraph) to construct the agent. 
Currently, we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.


Now, we can initialize the agent with the LLM and the tools.

Note that we are passing in the `model`, not `model_with_tools`. That is because `create_react_agent` will call `.bind_tools` for us under the hood.

In [61]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent

We can now run the agent with a few queries! Note that for now, these are all **stateless** queries (it won't remember previous interactions). Note that the agent will return the **final** state at the end of the interaction (which includes any inputs, we will see later on how to get only the outputs).

First up, let's see how it responds when there's no need to call a tool:

In [64]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='0157db79-ce15-4831-adc5-7e5cbc71121b'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run--2a494a22-6a02-4d02-86f5-089ad6a79609-0', usage_metadata={'input_tokens': 81, 'output_tokens': 10, 'total_tokens': 91, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In order to see exactly what is happening under the hood (and to make sure it's not calling a tool) we can take a look at the [LangSmith trace](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r)

Let's now try it out on an example where it should be invoking the tool

In [67]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='2a3fee30-0c74-4d01-8cbd-2d0cd9b794f8'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XymmZmZ4yygePl4OxjzNptCU', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b9eaab50-8785-4e9b-820e-81dbd065b0fa-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_XymmZmZ

We can check out the [LangSmith trace](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r) to make sure it's calling the search tool effectively.

## Streaming Messages

We've seen how the agent can be called with `.invoke` to get  a final response. If the agent executes multiple steps, this may take a while. To show intermediate progress, we can stream back messages as they occur.

In [71]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_v6cFgYp5fHc3fHcNaRmOizNd)
 Call ID: call_v6cFgYp5fHc3fHcNaRmOizNd
  Args:
    query: current weather in San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weather25.com/north-america/usa/california/san-francisco?page=month&month=September", "content": "# San Francisco weather in September 2025\n\nThe **temperatures in San Francisco in September are comfortable** with low of 14°C and and high up to 25°C.\n\nThere is little to no rain in San Francisco during September, so it’s a lot easier to explore the city. Just remember to dress in warm layers, as it can still get pretty chilly.\n\nOur weather forecast can give you a great sense of what weather to expect 

## Streaming tokens

In addition to streaming back messages, it is also useful to stream back tokens.
We can do this by specifying `stream_mode="messages"`.


::: note

Below we use `message.text()`, which requires `langchain-core>=0.3.37`.

:::

In [73]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| current| weather| in| San| Francisco| features| a| marine| layer|,| resulting| in| cool| and| mist|y| conditions|.| The| temperatures| are| around| |14|°C| (|57|°F|)| in| the| morning|,| with| a| high| expected| to| reach| approximately| |18|°C| (|64|°F|)| later| in| the| day|.| 

|For| more| detailed| forecasts| and| updates|,| you| can| check| the| following| resources|:| 
|-| [|Weather|25|](|https|://|www|.weather|25|.com|/n|orth|-amer|ica|/|usa|/cal|ifornia|/s|an|-fr|anc|isco|?page|=t|oday|)
|-| [|Sp|reak|er| Weather| Update|](|https|://|www|.sp|reak|er|.com|/|episode|/t|oday|-s|-weather|-in|-san|-fr|anc|isco|-|06|-|09|-|25|-m|arine|-layer|-m|agic|-and|-m|isty|-m|orning|-m|eteor|ological|-m|ay|hem|--|664|687|37|)| 

|Please| let| me| know| if| you| need| more| information|!|

## Adding in memory

As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a `thread_id` when invoking the agent (so it knows which thread/conversation to resume from).

In [77]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [79]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [81]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 83, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run--b6e5807e-cdfc-4d30-b935-2eb122a02b12-0', usage_metadata={'input_tokens': 83, 'output_tokens': 11, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [82]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 106, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run--78de8500-5ab4-465d-8aed-d2246a862cf4-0', usage_metadata={'input_tokens': 106, 'output_tokens': 13, 'total_tokens': 119, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


Example [LangSmith trace](https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)

If you want to start a new conversation, all you have to do is change the `thread_id` used

In [87]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I don't have access to personal data about users unless it has been shared with me in the course of our conversation. Therefore, I don’t know your name. If you tell me your name, I can use it in our chat!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 84, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run--b0b0fb72-94e0-460f-9e7f-c0cec1a994b3-0', usage_metadata={'input_tokens': 84, 'output_tokens': 48, 'total_tokens': 132, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


## Conclusion

That's a wrap! In this quick start we covered how to create a simple agent. 
We've then shown how to stream back a response - not only with the intermediate steps, but also tokens!
We've also added in memory so you can have a conversation with them.
Agents are a complex topic with lots to learn! 

For more information on Agents, please check out the [LangGraph](/docs/concepts/architecture/#langgraph) documentation. This has it's own set of concepts, tutorials, and how-to guides.